In [ ]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_SERIALIZABLE
from tabulate import tabulate

In [ ]:
def execute_trans(sql_commands):
  connection = psycopg2.connect(
      host= 'localhost',
      database= 'postgres',
      user= 'postgres',
      password= '1701'
  )
  connection.autocommit = False # We have turned off the auto commit to manage the transactions and to apply ACID properties
  cursor = connection.cursor()
  try :
    # Here we set the isolation level to serializabale
    connection.set_isolation_level(ISOLATION_LEVEL_SERIALIZABLE)

    # Now this will create a save point before any vital operations is performed
    cursor.execute('SAVEPOINT critical_operations')

    for sql_commands in sql_commands:
      cursor.execute(sql_commands)

    # will commit to the transaction if everything is successful
    connection.commit()
    print('transaction has been successful')
  except Exception as e:
    print(f"transaction failed: {e}")

    # This will roll back the transactions to the save point if any critical error has occured
    connection.rollback_to_savepoint("critical_operations")
    print("rolled back to saved point")

  finally:
    cursor.close()
    connection.close()


In [ ]:
#This will drop the table if they exist
drop_tables = ["""
DROP TABLE IF EXISTS stock;
DROP TABLE IF EXISTS product;
DROP TABLE IF EXISTS depot;
"""]
execute_trans(drop_tables)

transaction has been successful


In [ ]:
#Creatng the table without any constraints
create_tables = ["""

CREATE TABLE IF NOT EXISTS product (
    prod_id VARCHAR(5),
    pname VARCHAR(50),
    price DECIMAL(10, 2)
);

CREATE TABLE IF NOT EXISTS depot (
    dep_id VARCHAR(5),
    addr VARCHAR(50),
    volume INT
);

CREATE TABLE IF NOT EXISTS stock (
    prod_id VARCHAR(5),
    dep_id VARCHAR(5),
    quantity INT
);"""]

execute_trans(create_tables)

transaction has been successful


In [ ]:
# Adding the required constraints
alter_commands = ["""
ALTER TABLE Product ADD CONSTRAINT pk_product PRIMARY KEY (prod_id);
                  
ALTER TABLE Product ADD CONSTRAINT ck_product_price CHECK (price > 0);
                  
ALTER TABLE depot ADD CONSTRAINT pk_depot PRIMARY KEY (dep_id);
                  
ALTER TABLE stock ADD CONSTRAINT pk_stock PRIMARY KEY (prod_id,dep_id);
                  
ALTER TABLE stock ADD CONSTRAINT fk_product FOREIGN KEY (prod_id) REFERENCES product(prod_id) ON DELETE CASCADE ON UPDATE CASCADE;
                  
ALTER TABLE stock ADD CONSTRAINT fk_depot FOREIGN KEY (dep_id) REFERENCES depot(dep_id) ON DELETE CASCADE ON UPDATE CASCADE;
"""]
execute_trans(alter_commands)

transaction has been successful


In [ ]:
# Inserting values into the tables
insert_values = ["""
INSERT INTO product (prod_id, pname, price)
VALUES 
    ('p1', 'tape', 2.5),
    ('p2', 'tv', 250),
    ('p3', 'vcr', 80);

INSERT INTO depot (dep_id, addr, volume)
VALUES ('d1', 'NEW YORK', 9000),
       ('d2', 'SYRACUSE', 6000),
       ('d4', 'NEW YORK', 2000);

INSERT INTO stock (prod_id, dep_id, quantity)
VALUES 
    ('p1', 'd1', 1000),
    ('p1', 'd2', -100),
    ('p1', 'd4', 1200),
    ('p3', 'd1', 3000),
    ('p3', 'd4', 2000),
    ('p2', 'd4', 1500),
    ('p2', 'd1', -400),
    ('p2', 'd2', 2000);
"""]

execute_trans(insert_values)

transaction has been successful


In [ ]:
transaction_1 = [
    "DELETE FROM product WHERE prod_id='p1';"   
]

"ROLLBACK;" #can add roll back to revert all the changes made.

execute_trans(transaction_1)

transaction has been successful


In [ ]:
transaction_2 = [
    "DELETE FROM depot WHERE dep_id='d1';"   
]

"ROLLBACK;" #can add roll back to revert all the changes made.

execute_trans(transaction_2)

transaction has been successful


In [ ]:
transaction_3 = [
    "UPDATE depot SET dep_id = 'dd1' WHERE dep_id = 'd1';"   
]

"ROLLBACK;" #can add roll back to revert all the changes made.

execute_trans(transaction_3)

transaction has been successful


In [ ]:
transaction_4 = [
    "UPDATE product SET prod_id = 'pp1' WHERE prod_id = 'p1';"
]

"ROLLBACK;" #can add roll back to revert all the changes made.

execute_trans(transaction_4)

transaction has been successful


In [ ]:
transaction_5 = [
    "INSERT INTO product(prod_id,pname,price) VALUES ('p100', 'cd', 5);"
    "INSERT INTO stock(prod_id,dep_id,quantity) VALUES ('p100', 'd2', 50);"
]

"ROLLBACK;" #can add roll back to revert all the changes made.

execute_trans(transaction_5)

transaction has been successful


In [ ]:
transaction_6 = [
    "INSERT INTO depot VALUES ('d100', 'Chicago', 100);"
    "INSERT INTO stock VALUES ('p1', 'd100', 100);"
]

"ROLLBACK;" #can add roll back to revert all the changes made.

execute_trans(transaction_6)

transaction has been successful
